# Notes

This assignment is devoted to `pandas`. It covers indexing and filtering, and some `groupby` and `join` operations. The assignment roughly corresponds to Week 4 and the beginning of Week 5 of the course.

The main dataset you'll be using is [Titanic](https://www.kaggle.com/c/titanic). Please, note, that you must not rely on any specific location for the dataset, hence, any code like

```python
titanic_train = pd.read_csv("<location>/train.csv")
```

will fail and your notebook won't be validated and graded. Inputs to the functions are described explicitly in each case, and that's the only thing you can rely on.

In [39]:
%pylab inline
plt.style.use("bmh")

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [40]:
plt.rcParams["figure.figsize"] = (6,6)

In [41]:
import numpy as np
import pandas as pd

In [42]:
STUDENT = "Evgeniia Amineva, Doron Nissimi"
ASSIGNMENT = 4
TEST = False

In [43]:
if TEST:
    import solutions
    total_grade = 0
    MAX_POINTS = 21

In [44]:
if not TEST:
    titanic_train = pd.read_csv('train.csv')
    titanic_test = pd.read_csv('test.csv')
    titanic = pd.concat([titanic_train, titanic_test])
    print(titanic.head())

   PassengerId  Survived  Pclass  \
0            1       0.0       3   
1            2       1.0       1   
2            3       1.0       3   
3            4       1.0       1   
4            5       0.0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


# Indexing and filtering

### 1. Fixing age (1 point).

There are several known mistakes in the Titanic dataset.

Namely, [Julia Florence Siegel](https://www.encyclopedia-titanica.org/titanic-survivor/julia-florence-cavendish.html) (Mrs. Tyrell William Cavendish) is mistakenly marked as being 76 years old (the age she actually died, but many years after Titanic).

You must **replace the corresponding age value in the dataframe with her actual age at the time** (25) and return the dataset. Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return a copy of the input dataframe, and not perform replacement in the original dataframe. Structure and indexing must be the same as in input.

In [45]:
def fix_age(df):
    """Fix age for Julia Florence Siegel."""
    new_df = df.copy()
    new_df.loc[new_df['Name'].str.match(r'.*Cavendish, Mrs. Tyrell William.*'), 'Age'] = 25
    return new_df

In [46]:
if not TEST:
    new_df = fix_age(titanic)
    print(new_df[new_df['Name'].str.match(r'.*Cavendish, Mrs. Tyrell William.*')== True][['Name','Age']])
    print(titanic[titanic['Name'].str.match(r'.*Cavendish, Mrs. Tyrell William.*')== True][['Name', 'Age']])

                                                 Name   Age
96  Cavendish, Mrs. Tyrell William (Julia Florence...  25.0
                                                 Name   Age
96  Cavendish, Mrs. Tyrell William (Julia Florence...  76.0


In [47]:
PROBLEM_ID = 1

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

### 2. Embarkment port distribution (1 point).

You must find the value counts for embarkment port (`Embarked` column) for the passengers, who travelled in 3-d class, were male and between 20 and 30 years old (both inclusive). No need to treat missing values separately.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return **series**, indexed with values from `Embarked`, according to `.value_counts()` method semantics:

```
S    <number of male passengers in 3-d class, embarked at S, 20<=Age<=30>
C    <...>
Q    <...>
Name: Embarked, dtype: int64
```

In [48]:
def embarked_stats(df):
    """Calculate embarkment port statistics."""
    return pd.Series(df[(df['Pclass']==3) & (df['Sex']=='male') & (df['Age']>=20)&(df['Age']<=30)]['Embarked'].value_counts(), name='Embarked')

In [49]:
if not TEST:
    print(embarked_stats(titanic))

Embarked
S    132
C     21
Q      7
Name: Embarked, dtype: int64


In [50]:
PROBLEM_ID = 2

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, embarked_stats)

### 3. Fill missing age values (1 point).

Some age values are missing in the Titanic dataset. You need to calculate average age over all passengers, and fill missing age values in `Age` column.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure, but without missing values in `Age` column.

In [51]:
def fix_age(df):
    """Fix missing age values."""
    new_df = df.copy()
    new_df['Age'] = new_df['Age'].fillna(new_df['Age'].mean())
    return new_df

In [52]:
if not TEST:
    print(titanic['Age'].isna().sum())
    ndf = fix_age(titanic)
    print(ndf['Age'].isna().sum())

263
0


In [53]:
PROBLEM_ID = 3

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

### 4. Child travelling alone (1 point).

You must find a child (`Age<10`) on-board, who was travelling without siblings or parents and find a name of her nursemaid.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two strings, collected from `Name` column, with one being child's name and second being nursemaid's name. It's known, that there's **only one child** like this.

In [54]:
def get_nursemaid(df):
    ch_alone = df[(df['Age']<10)&(df['SibSp']==0)&(df['Parch']==0)]
    return tuple(df[df['Ticket']==ch_alone['Ticket'].iloc[0]]['Name'])

In [55]:
if not TEST:
    print(titanic[(titanic['Age']<10)&(titanic['SibSp']==0)&(titanic['Parch']==0)]['Name'])
    print(get_nursemaid(titanic))

777    Emanuel, Miss. Virginia Ethel
Name: Name, dtype: object
('Dowdell, Miss. Elizabeth', 'Emanuel, Miss. Virginia Ethel')


In [56]:
PROBLEM_ID = 4

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_nursemaid)

### 5. Port with the most children embarked (1 point).

You must find, which port had the largest percentage of children (`Age<10`) embarked, i.e. number of children divided by total number of passengers embarked.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **single string** with port letter.

In [57]:
def get_port(df):
    """Get port with the most children embarked."""
    return (df[df['Age']<10]['Embarked'].value_counts() / df['Embarked'].value_counts()).idxmax()

In [58]:
if not TEST:
    print(titanic[titanic['Age']<10]['Embarked'].value_counts() / titanic['Embarked'].value_counts())
    print(get_port(titanic))

Embarked
S    0.070022
C    0.051852
Q    0.032520
Name: count, dtype: float64
S


In [59]:
PROBLEM_ID = 5

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_port)

### 6. Passengers per ticket (2 points).

Calculate average and maximum number of passengers per ticket.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two values - average and maximum number of passengers per ticket.

In [60]:
def get_ticket_stats(df):
    """Calculate passenger per ticket statistics."""
    return (df['Ticket'].value_counts().mean(), df['Ticket'].value_counts().max())

In [64]:
if not TEST:
    print(get_ticket_stats(titanic))

(1.4090419806243273, 11)


In [65]:
PROBLEM_ID = 6

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_ticket_stats)

### 7. Fare per passenger (3 points).

For each individual ticket, you must calculate **fare per person for that ticket**, and then calculate averages for each class. Note, that you will need to apply `groupby` and you may consider using `.first()` of resulting `DataFrameGroupBy`. Also, caferully consider, in which order calculations are performed.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `pd.Series` with three elements, indexed by class:

```
1    <average per person fare in class 1>
2    <...>
3    <...>
Name: Pclass, dtype: float64
```

In [66]:
def get_fare_per_pass(df):
    """Calculate fare per passenger for different classes."""
    ndf = df.copy()
    # create column 'FPP' - fare per person
    ndf['FPP'] = ndf['Fare'] / ndf.groupby('Ticket')['PassengerId'].transform('count')
    # calculate avg fare per person for each class
    return pd.Series(ndf.groupby('Pclass')['FPP'].mean(), name='Pclass')

In [67]:
if not TEST:
    print(get_fare_per_pass(titanic))

Pclass
1    33.910500
2    11.411010
3     7.329146
Name: Pclass, dtype: float64


In [68]:
PROBLEM_ID = 7

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_fare_per_pass)

### 8. Fill missing age values (3 points).

In problem 3 you filled missing age values with global average over all passengers. Now, you need to fill them **according to class and sex**. For example, for a female passenger from 2d class, missing age value must be filled with average age of females in 2d class.

In this problem, you may need joins and `.apply()`, although there are several ways to get the same result.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure as input, but without missing values in `Age` column.

In [81]:
def fix_age_groupped(df):
    """Fill missing age values."""
    ndf = df.copy()
    # get over unique values of Sex 
    for s in ndf['Sex'].unique():
        # get over unique values of Pclass
        for c in ndf['Pclass'].unique():
            # apply a mask for each pair s and c and assign the mean values for such mask
            ndf.loc[(ndf['Sex']==s)&(ndf['Pclass']==c)&(ndf['Age'].isna()), 'Age'] = ndf[(ndf['Sex']==s)&(ndf['Pclass']==c)]['Age'].mean()
    return ndf

In [82]:
if not TEST:
    # check missing values
    print(titanic['Age'].isna().sum())
    # check mean value by Sex and by Plass
    print(titanic.groupby(['Sex', 'Pclass'])['Age'].mean())
    # apply the function
    ndf = fix_age_groupped(titanic)
    # check missing values in the new df
    print(ndf['Age'].isna().sum())
    # check mean value by Sex and by Plass in new df
    print(ndf.groupby(['Sex', 'Pclass'])['Age'].mean())

263
Sex     Pclass
female  1         37.037594
        2         27.499223
        3         22.185329
male    1         41.029272
        2         30.815380
        3         25.962264
Name: Age, dtype: float64
0
Sex     Pclass
female  1         37.037594
        2         27.499223
        3         22.185329
male    1         41.029272
        2         30.815380
        3         25.962264
Name: Age, dtype: float64


In [ ]:
PROBLEM_ID = 8

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age_groupped)

### 9. Finding couples (3 points).

Based on the code from Lecture 5, build a dataframe of couples. Filter it by survival status: select those couples, in which only one of spouses survived or none of two. Built survival statistics by class, i.e. ratio of the number couples with partial survival or couples which died together, divided by total number of couples in class. If the survival status of one or both of spouses is not known, it must be considered as `0`.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `Series` with three elements indexed by values from `Pclass` column (see P7 as a reference).

In [ ]:
def find_couples(df):
    """Your code here."""
    pass

In [ ]:
PROBLEM_ID = 9

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, find_couples)

### 10. Lonely Passengers (2 points).

Extract # of passengers per class who were either traveling alone (no siblings/spouses/parents/children and also bought a ticket for one) or they have a ticket number beginning with "P", **but not both**.

Note that passenger traveling alone who has a ticket "PC 1234" **should NOT be counted**, but a passenger traveling alone with a ticket "AC 11" should be counted.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**.\
Output must be `Series` with three elements indexed by values from `Pclass` column.

In [78]:
def lonely_or_p(df):
    """Your code here."""
    # get list of tickets, which were bought for one
    tickets = titanic['Ticket'].value_counts()
    tickets_fo = tickets[tickets==1].index

    # apply filters
    ndf = titanic[((titanic['SibSp']==0)&(titanic['Parch']==0)&(titanic['Ticket'].str.match(r'^[^P].*'))&(titanic['Ticket'].isin(tickets_fo)))\
    |((titanic['Ticket'].str.match(r'^[P].*'))&((~titanic['Ticket'].isin(tickets_fo))|(titanic['SibSp']!=0)|(titanic['Parch']!=0)))]

    return ndf.groupby('Pclass').size()

In [80]:
if not TEST:
    print(lonely_or_p(titanic))

Pclass
1    143
2    130
3    438
dtype: int64


In [ ]:
PROBLEM_ID = 10

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, lonely_or_p)

### 11. Family Tickets (3 points).

Find for each class a proportion of family tickets - tickets where all passengers have the same last name.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `Series` with three elements indexed by values from `Pclass` column, with values between 0 and 1, containing the proportion of family tickets for that class.

In [141]:
def family_tickets(df):
    """Your code here."""
    # create a copy of df
    ndf = df.copy()
    # split the column Name
    ndf[['fam_name', 'main_name']] = ndf.Name.str.split(",", expand=True)
    # count members of one family in each ticket
    ntdf = ndf.groupby(['Ticket', 'fam_name'])['Pclass'].value_counts()
    # get only tickets where more than one person have the same family name
    ttdf = ntdf[ntdf>1].to_frame().reset_index()
    # return a proportion of family tickets in each class
    return ttdf.groupby('Pclass').size() / df.groupby('Pclass').size()

In [149]:
if not TEST:
    # let's implement a part of the code above and make sure that there isn't more that one members of different families in one ticket
    ndf = titanic.copy()
    # split the column Name
    ndf[['fam_name', 'main_name']] = ndf.Name.str.split(",", expand=True)
    # count members of one family in each ticket
    ntdf = ndf.groupby(['Ticket', 'fam_name'])['Pclass'].value_counts()
    # get only tickets where more than one person have the same family name
    ttdf = ntdf[ntdf>1].to_frame().reset_index()
    #ttdf = ntdf.to_frame().reset_index()
    # check number of families after an implementation of filter >1
    print(ttdf.groupby(['Ticket'])['fam_name'].count().sort_values(ascending=False))   
    # check the function
    print(family_tickets(titanic))

Ticket
110413       1
110813       1
36973        1
370129       1
370365       1
            ..
2653         1
2659         1
2660         1
2661         1
WE/P 5735    1
Name: fam_name, Length: 176, dtype: int64
Pclass
1    0.188854
2    0.166065
3    0.097320
dtype: float64


In [ ]:
PROBLEM_ID = 11

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, family_tickets)

# Your grade

In [ ]:
if TEST:
    print(f"{STUDENT}: {int(100 * total_grade / MAX_POINTS)}")